In [1]:
# !pip install datasets
from datasets import load_dataset, DatasetDict, Dataset

c:\Users\ADMIN\miniconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!pip install evaluate

  Using cached evaluate-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached responses-0.18.0-py3-none-any.whl.metadata (29 kB)
Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
Using cached responses-0.18.0-py3-none-any.whl (38 kB)


In [2]:
# !pip install transformers
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

In [3]:
# ! pip install peft
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [6]:
import torch
torch.cuda.is_available()

True

In [4]:
model_checkpoint = 'LiyaT3/sentiment-analysis-imdb-distilbert'

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)
# for name, param in model.named_parameters():
#     param.requires_grad = False

In [5]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [6]:
for name, param in model.named_parameters():
     if name.startswith("classifier"): # choose whatever you like here
        param.requires_grad = True

In [7]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_autoset_attn_implementation',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_check_and_enable_sdpa',
 '_compiled_call_impl',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_expand_inputs_for_generation',
 '_extract_past_from_model_output',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_k

In [8]:
from datasets import load_dataset

dataset = load_dataset("sg247/binary-classification")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8004
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})


In [9]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df.dropna(inplace = True)


df['label'] = df['label'].astype(int)
print(df)


                                                  tweet  label
0     Want to say a huge thanks to @WarriorAssaultS ...      1
1     @jaynehh_ you just need a job and get a letter...      1
2             @knhillrocks HA yes, make it quick tho :D      1
3     @shartyboy Thanks for texting me back :)) I'm ...      1
4     Laying out a greetings card range for print to...      1
...                                                 ...    ...
7999  "@aula_jr: @Jude_Mugabi I'm watching it. 2-0 M...      0
8000                                  why so sudden :((      0
8001        @21oclock :((( bout to instant transmission      0
8002  @ButtCupboard my feet were cold!!!! and there ...      0
8003  @Kimwoobin89__ oppa............./hug ur arm/ w...      0

[8000 rows x 2 columns]


In [10]:
d = pd.DataFrame(dataset['test'])
d.dropna(inplace = True)
d['label'] = d['label'].astype(int)


In [11]:
from datasets import Dataset, DatasetDict
train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}
train_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'train': train_dataset})
print(dataset_dict['train']['label'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [12]:
from datasets import Dataset, DatasetDict
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}
test_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'test': train_dataset})

# print(dataset_dict['train']['label'][2])
print(dataset_dict['test']['label'][2])
print(dataset_dict)

1
DatasetDict({
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
})


In [13]:
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}

train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}



import datasets
train_dataset = datasets.Dataset.from_dict(train_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)
my_dataset_dict = datasets.DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})
print(my_dataset_dict)
print(my_dataset_dict['test']['label'][2])
print(my_dataset_dict['train']['label'][2])

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})
1
1


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    tweets = examples["tweet"]
    preprocess_t= [tweet if isinstance(tweet , str) else str(tweet) for tweet in tweets]
    return tokenizer(preprocess_t , truncation= True , padding= "max_length")

tokenized_dataset = my_dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 2000/2000 [00:00<00:00, 5193.52 examples/s]


DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [23]:
torch.cuda.empty_cache()


In [18]:
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
     if name.startswith("classifier"):
        param.requires_grad = True

In [24]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# training 

ranks = [4,8,20]

for r in ranks:
    print(r)
    peft_config = LoraConfig(task_type="SEQ_CLS",
                            r=r,
                            lora_alpha=32,
                            lora_dropout=0.01,
                            target_modules = ['q_lin','k_lin'])

    peft_config

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    lr = 0.001
    batch_size = 16
    num_epochs = 1
    training_args = TrainingArguments(
        output_dir= model_checkpoint + "-lora-text-classification",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )
    accuracy = evaluate.load("accuracy")
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return accuracy.compute(predictions=predictions, references=labels)


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        eval_dataset = tokenized_dataset['train'],
        compute_metrics=compute_metrics, 
    )
    trainer.train()

4
trainable params: 665,858 || all params: 67,620,868 || trainable%: 0.9846930684178736


c:\Users\ADMIN\miniconda3\envs\tf_gpu\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
                                                
 10%|█         | 50/500 [28:21<02:15,  3.32it/s] 

{'loss': 0.0212, 'learning_rate': 0.0, 'epoch': 1.0}





















































































































































































































































































































































































































































































































                                                

                                           
 10%|█         | 50/500 [29:33<02:15,  3.32it/s] 

Checkpoint destination directory LiyaT3/sentiment-analysis-imdb-distilbert-lora-text-classification\checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
                                                
100%|██████████| 500/500 [03:43<00:00,  2.24it/s]


{'eval_loss': 0.0031875313725322485, 'eval_accuracy': 0.999375, 'eval_runtime': 71.0671, 'eval_samples_per_second': 112.57, 'eval_steps_per_second': 7.036, 'epoch': 1.0}
{'train_runtime': 223.6335, 'train_samples_per_second': 35.773, 'train_steps_per_second': 2.236, 'train_loss': 0.02116710090637207, 'epoch': 1.0}
8
trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925332946813067


...

{'eval_loss': 0.002690572990104556, 'eval_accuracy': 0.9995, 'eval_runtime': 72.4696, 'eval_samples_per_second': 110.391, 'eval_steps_per_second': 6.899, 'epoch': 1.0}
{'train_runtime': 226.4168, 'train_samples_per_second': 35.333, 'train_steps_per_second': 2.208, 'train_loss': 0.020570178985595704, 'epoch': 1.0}
20
trainable params: 960,770 || all params: 67,915,780 || trainable%: 1.4146491433949517


c:\Users\ADMIN\miniconda3\envs\tf_gpu\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
                                                
 10%|█         | 50/500 [36:01<02:15,  3.32it/s] 

{'loss': 0.0239, 'learning_rate': 0.0, 'epoch': 1.0}





















































































































































































































































































































































































































































































































                                                

                                           
 10%|█         | 50/500 [37:15<02:15,  3.32it/s] 

Checkpoint destination directory LiyaT3/sentiment-analysis-imdb-distilbert-lora-text-classification\checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
                                                
100%|██████████| 500/500 [03:48<00:00,  2.19it/s]

{'eval_loss': 0.003640302922576666, 'eval_accuracy': 0.99925, 'eval_runtime': 73.5463, 'eval_samples_per_second': 108.775, 'eval_steps_per_second': 6.798, 'epoch': 1.0}
{'train_runtime': 228.5311, 'train_samples_per_second': 35.006, 'train_steps_per_second': 2.188, 'train_loss': 0.023881811141967774, 'epoch': 1.0}


In [16]:
!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl.metadata (58 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl (9.3 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl (42.2 MB)
Using cached threadpoolctl-3.4.0-py3-none-any.whl (17 kB)


In [39]:
trainer.save_model("Trained_Model_LoRA")

In [40]:
from transformers import AutoModelForSequenceClassification

path_to_saved_model = "Trained_Model_LoRA"
loaded_model = AutoModelForSequenceClassification.from_pretrained(path_to_saved_model)


In [48]:
inputs = tokenizer.encode("You are ugly", return_tensors="pt")
logits = loaded_model(inputs).logits
predictions = torch.max(logits,1).indices
print(predictions)
print("you are ugly" + " - " + id2label[predictions.tolist()[0]])

tensor([0])
you are ugly - Negative
